In [ ]:
# !pip install -q tensorflow-recommenders
# !pip install -q --upgrade tensorflow-datasets
# !pip install -q scann

In [ ]:
import pandas as pd
import requests
import os
import gzip
import json
import numpy as np

import pprint
import tempfile

from typing import Dict, Text

import tensorflow as tf
# from tfx import v1 as tfx

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [ ]:
# ratings_df = pd.read_csv('/content/drive/MyDrive/KaggleX/model_data/ratings/all_ratings.csv')
ratings_df = pd.read_csv('/content/drive/MyDrive/KaggleX/model_data/ratings/read_ratings.csv', names=['user_id', 'book_id'])


In [ ]:
# books_df = pd.read_csv('/content/drive/MyDrive/KaggleX/model_data/books/all_books.csv')
books_df = pd.read_csv('/content/drive/MyDrive/KaggleX/model_data/books/read_books.csv', names=['title', 'book_id','ratings_count'])


In [ ]:
books_df.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 2360656 entries, nan to 2360654.0
Data columns (total 3 columns):
 #   Column         Dtype  
---  ------         -----  
 0   title          object 
 1   book_id        int64  
 2   ratings_count  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 72.0+ MB


In [ ]:
ratings_df = ratings_df.merge(books_df, how='inner', on='book_id')

In [ ]:
# ratings_df.sort_values(by='ratings_count', ascending=False, inplace=True)

In [ ]:
books_df.head()

,title,book_id,ratings_count
NaN,0,1,2.0
0.0,W.C. Fields: A Life on Film,5333265,3.0
1.0,Good Harbor,1333909,10.0
2.0,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",7327624,140.0
3.0,Best Friends Forever,6066819,51184.0


In [ ]:
ratings_df = ratings_df[['book_id', 'user_id', 'title']]

In [ ]:
ratings_df = ratings_df.head(400000)

In [ ]:
ratings_df.to_csv('/content/drive/MyDrive/KaggleX/model_data/ratings/pipeline_ratings/ratings_w_title.csv', index=False)

In [ ]:
# books_df = books_df[['title', 'popular_shelves', 'description', 'authors']]
print(len(books_df))
books_df = books_df.merge(ratings_df, how='inner', on='book_id')

# books_df = books_df[['title']]

2360656


In [ ]:
books_df.head()
books_df = books_df['title_y'].unique()
# books_df = c.to_frame(name="title_y")
print(len(books_df))
# books_df.to_csv()

29


In [ ]:
books_df = pd.DataFrame(books_df, columns =['title'])

In [ ]:
books_df.head()

,title
0,0
1,Harry Potter and the Half-Blood Prince (Harry ...
2,The Name of the Wind (The Kingkiller Chronicle...
3,Open
4,The Case Against Sugar


In [ ]:
# books_df.to_csv('/content/drive/MyDrive/KaggleX/model_data/books/title_index.csv', index=False)
books_df.to_csv('/content/drive/MyDrive/KaggleX/model_data/books/pipeline_books/book_title.csv', index=False)

In [ ]:
ratings_df.fillna('', inplace=True)
books_df.fillna('', inplace=True)

In [ ]:
ratings_df.head()

,book_id,user_id,title
0,1,0,0
1,1,0,Harry Potter and the Half-Blood Prince (Harry ...
2,1,8842281e1d1347389f2ab93d60773d4d,0
3,1,8842281e1d1347389f2ab93d60773d4d,Harry Potter and the Half-Blood Prince (Harry ...
4,1,72fb0d0087d28c832f15776b0d936598,0


In [ ]:
ratings = tf.data.Dataset.from_tensor_slices((dict(ratings_df)))
books = tf.data.Dataset.from_tensor_slices((dict(books_df)))

In [ ]:
books

<_TensorSliceDataset element_spec={'title': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [ ]:
ratings = ratings.map(lambda x: {
    "title": x["title"],
    "user_id": x["user_id"],

})
books = books.map(lambda x: x["title"])

In [ ]:
books

<_MapDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [ ]:
unique_book_titles = np.unique(np.concatenate(list(books.batch(1000))))

unique_user_ids = np.unique(np.concatenate(list(ratings.batch(1_000).map(
    lambda x: x["user_id"]))))

In [ ]:
unique_user_ids

array([b'0', b'00000377eea48021d3002730d56aca9a',
       b'000079c580bbe45e1500acabe551b276', ...,
       b'ffff7cafdaf5196383cb2efca08fb6fe',
       b'ffffbb062a8b208c9c1031b529c08f7a',
       b'fffff8a718843c0e11dfd93fb41c1297'], dtype=object)

In [ ]:
embedding_dimension = 32

In [ ]:
class UserModel(tf.keras.Model):

  def __init__(self):
    super().__init__()


    self.user_id_embedding = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=unique_user_ids, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_ids) + 1, 32),
    ])


  def call(self, inputs):
    # Take the input dictionary, pass it through each input layer,
    # and concatenate the result.
    return tf.concat([
        self.user_id_embedding(inputs["user_id"]),
    ], axis=1)

In [ ]:
class QueryModel(tf.keras.Model):
  """Model for encoding user queries."""

  def __init__(self, layer_sizes):
    """Model for encoding user queries.

    Args:
      layer_sizes:
        A list of integers where the i-th entry represents the number of units
        the i-th layer contains.
    """
    super().__init__()

    # We first use the user model for generating embeddings.
    self.embedding_model = UserModel()

    # Then construct the layers.
    self.dense_layers = tf.keras.Sequential()

    # Use the ReLU activation for all but the last layer.
    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

    # No activation for the last layer.
    for layer_size in layer_sizes[-1:]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))

  def call(self, inputs):
    feature_embedding = self.embedding_model(inputs)
    return self.dense_layers(feature_embedding)

In [ ]:
class BookModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    max_tokens = 10_000

    self.title_embedding = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_book_titles,mask_token=None),
      tf.keras.layers.Embedding(len(unique_book_titles) + 1, embedding_dimension)
    ])

    self.title_vectorizer = tf.keras.layers.TextVectorization(
        max_tokens=max_tokens)

    self.title_text_embedding = tf.keras.Sequential([
      self.title_vectorizer,
      tf.keras.layers.Embedding(max_tokens, embedding_dimension, mask_zero=True),
      tf.keras.layers.GlobalAveragePooling1D(),
    ])

    self.title_vectorizer.adapt(books)

  def call(self, titles):
    return tf.concat([
        self.title_embedding(titles),
        self.title_text_embedding(titles),
    ], axis=1)


In [ ]:
class CandidateModel(tf.keras.Model):
  """Model for encoding movies."""

  def __init__(self, layer_sizes):
    """Model for encoding movies.

    Args:
      layer_sizes:
        A list of integers where the i-th entry represents the number of units
        the i-th layer contains.
    """
    super().__init__()

    self.embedding_model = BookModel()

    # Then construct the layers.
    self.dense_layers = tf.keras.Sequential()

    # Use the ReLU activation for all but the last layer.
    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

    # No activation for the last layer.
    for layer_size in layer_sizes[-1:]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))

  def call(self, inputs):
    feature_embedding = self.embedding_model(inputs)
    return self.dense_layers(feature_embedding)

In [ ]:
class GoodreadsModel(tfrs.models.Model):

  def __init__(self, layer_sizes):
    super().__init__()
    self.query_model = QueryModel(layer_sizes)
    self.candidate_model = CandidateModel(layer_sizes)
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=books.batch(128).map(self.candidate_model),
        ),
    )

  def compute_loss(self, features, training=False):

    query_embeddings = self.query_model({
        "user_id": features["user_id"],
    })
    book_embeddings = self.candidate_model(features["title"])

    print('query embedding')
    print(query_embeddings)
    print('book_embedding')
    print(book_embeddings)

    return self.task(
        query_embeddings, book_embeddings, compute_metrics=not training)

In [ ]:
@tf.function
#  (input_signature=[tf.TensorSpec(shape=None, dtype=tf.int32)])
def serve_fn(model, input_data):
    # output_data = 'a thin'
    # tf.cast(input_data, tf.int64)
    layer_index = input_data[0]
    output_data = model.candidate_model.embedding_model.weights[0][layer_index]
    # model(input_data)
    # output_data = tf.nn.softmax(output_data)
    return {'outputs': output_data}

In [ ]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(32_000_000)
test = shuffled.skip(32_000_000).take(8_000_000)

cached_train = train.shuffle(100_000).batch(2048)
cached_test = test.batch(4096).cache()

In [ ]:
# num_epochs = 1

model = GoodreadsModel([64, 32])
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))


In [ ]:
one_layer_history = model.fit(
    cached_train,
    epochs=1)

# accuracy = one_layer_history.history["val_factorized_top_k/top_100_categorical_accuracy"][-1]
# print(f"Top-100 accuracy: {accuracy:.2f}.")

query embedding
Tensor("query_model_1/sequential_6/dense_5/BiasAdd:0", shape=(None, 32), dtype=float32)
book_embedding
Tensor("candidate_model_1/sequential_9/dense_7/BiasAdd:0", shape=(None, 32), dtype=float32)
query embedding
Tensor("query_model_1/sequential_6/dense_5/BiasAdd:0", shape=(None, 32), dtype=float32)
book_embedding
Tensor("candidate_model_1/sequential_9/dense_7/BiasAdd:0", shape=(None, 32), dtype=float32)
120/196 [=================>............] - ETA: 3s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 15954.6176 - regularization_loss: 0.0000e+00 - total_loss: 15954.6176

KeyboardInterrupt: ignored

In [ ]:
model.candidate_model.save("/content/drive/MyDrive/KaggleX/models/candidate_model7", save_format='tf')
# loaded_model = tf.keras.saving.load_model("/content/drive/MyDrive/KaggleX/models/candidate_model6")

In [ ]:
# model.evaluate(cached_test, return_dict=True)

In [ ]:
# loaded_model.predict(["The Three-Body Problem Remembrance of Earth's Past #1"])

In [ ]:
# model.build()

In [ ]:
model.candidate_model.summary()

In [ ]:
# model.candidate_model.predict(["The Three-Body Problem Remembrance of Earth's Past #1"])

In [ ]:
module = tf.Module()
module.model = model.candidate_model
module.serve = serve_fn
tf.saved_model.save(module,
                      "/content/drive/MyDrive/KaggleX/models/candidate_model_signature_def3",
                      signatures=serve_fn.get_concrete_function(
                      input_data=tf.TensorSpec(shape=(None,), dtype=tf.int64)
                      ),
                      options=None
                    )

In [ ]:
model.candidate_model.embedding_model.weights[0][44]

In [ ]:
loaded_model = tf.saved_model.load("/content/drive/MyDrive/KaggleX/models/candidate_model_signature_def3")

In [ ]:
loaded_model.signatures['serving_default']

In [ ]:
infer = loaded_model.signatures['serving_default']

In [ ]:
input_data = tf.constant([44], dtype=tf.int64)
predictions = infer(input_data)

In [ ]:
predictions

In [ ]:
! saved_model_cli show --all  --dir /content/drive/MyDrive/KaggleX/models/candidate_model_signature_def2

In [ ]:
import pickle

with open('/content/drive/MyDrive/KaggleX/candidate_model_vocab.pkl', 'wb') as f:
    pickle.dump(unique_book_titles, f)

In [ ]:
import pickle

with open('/content/drive/MyDrive/KaggleX/candidate_model_embeddings.pkl', 'wb') as f:
    pickle.dump(model.candidate_model.embedding_model.weights[0], f)